# Squad404

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import StratifiedKFold




### Loading the augmented images and converting to pd dataframe

In [4]:
# Reading the csv and images 
data_annotations_csv = 'Data - Needs Respray - 2024-03-26/Labels-NeedsRespray-2024-03-26.csv'
augmented_images = 'Data - Needs Respray - 2024-03-26'

data_file = pd.read_csv(data_annotations_csv, dtype={'label': str})


In [7]:
print(data_file)

                Filename Needs Respray
0   paver weeds - 03.png           Yes
1   paver weeds - 09.png           Yes
2   paver weeds - 17.png           Yes
3   paver weeds - 25.png            No
4   paver weeds - 32.png            No
5   paver weeds - 36.png            No
6   paver weeds - 43.png            No
7   paver weeds - 47.png            No
8   paver weeds - 59.png            No
9   paver weeds - 63.png           Yes
10  paver weeds - 77.png           Yes
11  paver weeds - 85.png           Yes


In [8]:
# Creating vectors of filenames and labels matched by index 
img_filenames_vector = data_file['Filename'].values
labels_vector = data_file['Needs Respray'].values

### Creating our model

Add prototypical network to our model before running it through the training

In [9]:
# Function to create a model from the pretrained CNN base you pass in
# Example usage: model = create_model(MobileNetv2)
def create_model(base):
    base_model = base(input_shape=(224, 224, 3), include_top=False)
    base_model.trainable = False # Freeze the base_model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


### Setting up k-fold

In [10]:
k = 5
kf = StratifiedKFold(n_splits=k, shuffle=True)

In [11]:
for train_index, val_index in kf.split(img_filenames_vector, labels_vector):
    print(f"train: {train_index}, val: {val_index}")

train: [ 0  1  2  3  4  5  6  7 11], val: [ 8  9 10]
train: [ 0  1  4  5  6  8  9 10 11], val: [2 3 7]
train: [ 1  2  3  4  5  7  8  9 10 11], val: [0 6]
train: [ 0  2  3  5  6  7  8  9 10 11], val: [1 4]
train: [ 0  1  2  3  4  6  7  8  9 10], val: [ 5 11]


In [12]:
CNN_scores = []
CNN_models_to_test = [MobileNetV2, ResNet50]

In [14]:
for CNN_base_model in CNN_models_to_test:
    fold_scores_for_CNN = []
    fold_no = 1
    print(f"TRAINING FOR {CNN_base_model}")
    for train_index, val_index in kf.split(img_filenames_vector, labels_vector):
        print(f"Fold number: {fold_no}/5")
        train_filenames, val_filenames = img_filenames_vector[train_index], img_filenames_vector[val_index]
        train_labels, val_labels = labels_vector[train_index], labels_vector[val_index]
        
        # Create ImageDataGenerator for train and validation
        train_datagen = ImageDataGenerator(rescale=1./255)
        val_datagen = ImageDataGenerator(rescale=1./255)
        
        # Create generators
        train_generator = train_datagen.flow_from_dataframe(
            dataframe=data_file.iloc[train_index],
            directory=augmented_images,
            x_col='Filename',
            y_col='Needs Respray',
            target_size=(224, 224),
            batch_size=16,
            class_mode='binary'
        )
        
        val_generator = val_datagen.flow_from_dataframe(
            dataframe=data_file.iloc[val_index],
            directory=augmented_images,
            x_col='Filename',
            y_col='Needs Respray',
            target_size=(224, 224),
            batch_size=16,
            class_mode='binary'
        )
        model = create_model(CNN_base_model)
        history = model.fit(train_generator, validation_data=val_generator, epochs=5)
        fold_scores_for_CNN.append(model.evaluate(val_generator))
        fold_no += 1
    CNN_scores.append(fold_scores_for_CNN)

TRAINING FOR <function MobileNetV2 at 0x2a0288860>
Fold number: 1/5
Found 9 validated image filenames belonging to 2 classes.
Found 3 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4444 - loss: 0.7442 - val_accuracy: 0.3333 - val_loss: 2.5632
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 563ms/step - accuracy: 0.8889 - loss: 0.2567 - val_accuracy: 0.3333 - val_loss: 2.3856
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 564ms/step - accuracy: 1.0000 - loss: 0.0419 - val_accuracy: 0.3333 - val_loss: 2.1931
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step - accuracy: 1.0000 - loss: 0.0257 - val_accuracy: 0.6667 - val_loss: 2.1819
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 565ms/step - accuracy: 1.0000 - loss: 0.0119 - val_accuracy: 0.6667 - val_loss: 2.2429
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.6667 - loss: 2.2429
Fold number: 2/5
Found 9 validated image filenames belonging to 2 classes.
Found 3 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.3333 - loss: 0.8059 - val_accuracy: 0.3333 - val_loss: 1.6115
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step - accuracy: 1.0000 - loss: 0.2427 - val_accuracy: 0.6667 - val_loss: 1.5103
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step - accuracy: 0.8889 - loss: 0.1725 - val_accuracy: 0.6667 - val_loss: 2.0051
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 560ms/step - accuracy: 1.0000 - loss: 0.0607 - val_accuracy: 0.6667 - val_loss: 2.5895
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step - accuracy: 1.0000 - loss: 0.0137 - val_accuracy: 0.6667 - val_loss: 3.1265
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.6667 - loss: 3.1265
Fold number: 3/5
Found 10 validated image filenames belonging to 2 classes.
Found 2 validated image filenames belonging to 2 classes.
Epoch 1/5


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 0.8953 - val_accuracy: 0.5000 - val_loss: 1.5667
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 762ms/step - accuracy: 0.5000 - loss: 1.5430 - val_accuracy: 0.5000 - val_loss: 1.0192
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 758ms/step - accuracy: 0.5000 - loss: 0.9832 - val_accuracy: 0.5000 - val_loss: 0.6951
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 762ms/step - accuracy: 0.9000 - loss: 0.5845 - val_accuracy: 0.5000 - val_loss: 1.0684
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 756ms/step - accuracy: 0.5000 - loss: 0.8611 - val_accuracy: 0.5000 - val_loss: 1.1171
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - accuracy: 0.5000 - loss: 1.1171
Fold number: 4/5
Found 10 validated image filenames belonging to 2 classes.
Found 2 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.7128 - val_accuracy: 0.5000 - val_loss: 1.4612
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 761ms/step - accuracy: 0.5000 - loss: 1.5271 - val_accuracy: 1.0000 - val_loss: 0.4661
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 765ms/step - accuracy: 0.8000 - loss: 0.5984 - val_accuracy: 0.5000 - val_loss: 0.9317
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 769ms/step - accuracy: 0.5000 - loss: 1.1295 - val_accuracy: 0.5000 - val_loss: 0.8111
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 763ms/step - accuracy: 0.5000 - loss: 1.0350 - val_accuracy: 1.0000 - val_loss: 0.4032
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - accuracy: 1.0000 - loss: 0.4032
Fold number: 5/5
Found 10 validated image filenames belonging to 2 classes.
Found 2 validated image filenames belonging to 2 classes.
Epoch 1/5


/Users/lukewang/Documents/COMP9417/Squad404/venv/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5000 - loss: 0.8937 - val_accuracy: 0.5000 - val_loss: 1.6799
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 773ms/step - accuracy: 0.5000 - loss: 1.7154 - val_accuracy: 0.5000 - val_loss: 1.1335
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 772ms/step - accuracy: 0.5000 - loss: 0.9439 - val_accuracy: 0.5000 - val_loss: 0.7245
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 771ms/step - accuracy: 0.5000 - loss: 0.6317 - val_accuracy: 0.5000 - val_loss: 0.9253
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 778ms/step - accuracy: 0.5000 - loss: 0.9444 - val_accuracy: 0.5000 - val_loss: 0.8275
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.5000 - loss: 0.8275


In [11]:
for score in CNN_scores:
    print(np.mean(score, axis=0))

[0.04338196 0.99375   ]
[0.7572655 0.50625  ]
